# Tavily Search

[Tavily's Search API](https://tavily.com) is a search engine built specifically for AI agents (LLMs), delivering real-time, accurate, and factual results at speed.

## Overview

### Integration details
| Class | Package | Serializable | [JS support](https://js.langchain.com/v0.2/docs/integrations/tools/tavily_search) |  Package latest |
| :--- | :--- | :---: | :---: | :---: |
| [TavilySearchResults](https://api.python.langchain.com/en/latest/tools/langchain_community.tools.tavily_search.tool.TavilySearchResults.html) | [langchain-community](https://api.python.langchain.com/en/latest/community_api_reference.html) | ❌ | ✅ |  ![PyPI - Version](https://img.shields.io/pypi/v/langchain-community?style=flat-square&label=%20) |

### Tool features
| [Returns artifact](/docs/how_to/tool_artifacts/) | Native async | Return data | Pricing |
| :---: | :---: | :---: | :---: |
| ✅ | ✅ | Title, URL, content, answer | 1,000 free searches / month | 


## Setup

The integration lives in the `langchain-community` package. We also need to install the `tavily-python` package.

In [1]:
%pip install --quiet -U langchain-community tavily-python

Note: you may need to restart the kernel to use updated packages.


### Credentials

We also need to set our Tavily API key. You can get an API key by visiting [this site](https://app.tavily.com/sign-in) and creating an account.

In [2]:
import getpass
import os

if not os.environ.get("TAVILY_API_KEY"):
    os.environ["TAVILY_API_KEY"] = getpass.getpass("Tavily API key:\n")

It's also helpful (but not needed) to set up [LangSmith](https://smith.langchain.com/) for best-in-class observability:

In [3]:
# os.environ["LANGCHAIN_TRACING_V2"] = "true"
# os.environ["LANGCHAIN_API_KEY"] = getpass.getpass()

## Instantiation

Here we show how to instatiate an instance of the Tavily search tools, with 

In [4]:
from langchain_community.tools import TavilySearchResults

tool = TavilySearchResults(
    max_results=5,
    search_depth="advanced",
    include_answer=True,
    include_raw_content=True,
    include_images=True,
    # include_domains=[...],
    # exclude_domains=[...],
    # name="...",            # overwrite default tool name
    # description="...",     # overwrite default tool description
    # args_schema=...,       # overwrite default args_schema: BaseModel
)

## Invocation

### [Invoke directly with args](/docs/concepts/#invoke-with-just-the-arguments)

The `TavilySearchResults` tool takes a single "query" argument, which should be a natural language query:

In [7]:
tool.invoke({"query": "What happened at the last wimbledon"})

[{'url': 'https://www.nytimes.com/athletic/5636272/2024/07/14/carlos-alcaraz-novak-djokovic-wimbledon-final/',
  'content': 'WIMBLEDON — Carlos Alcaraz beat Novak Djokovic in the Wimbledon final at the All England Club 6-2, 6-2, 7-6 on Sunday. The No 3 seed prevailed over the No 2 seed in a one-sided victory ...'},
 {'url': 'https://www.theguardian.com/sport/live/2023/jul/16/wimbledon-mens-singles-final-2023-carlos-alcaraz-v-novak-djokovic-live?page=with:block-64b3ff568f08df28470056bf',
  'content': 'Carlos Alcaraz recovered from a set down to topple Djokovic 1-6, 7-6(6), 6-1, 3-6, 6-4 and win his first Wimbledon title in a battle for the ages'},
 {'url': 'https://www.nytimes.com/athletic/live-blogs/wimbledon-2024-live-updates-alcaraz-djokovic-mens-final-result/kJJdTKhOgkZo/',
  'content': "It was Djokovic's first straight-sets defeat at Wimbledon since the 2013 final, when he lost to Andy Murray. Below, The Athletic 's writers, Charlie Eccleshare and Matt Futterman, analyze the ..."},

### [Invoke with ToolCall](/docs/concepts/#invoke-with-toolcall)

We can also invoke the tool with a model-generated ToolCall, in which case a ToolMessage will be returned:

In [9]:
# This is usually generated by a model, but we'll create a tool call directly for demon purposes.
model_generated_tool_call = {
    "args": {"query": "euro 2024 host nation"},
    "id": "1",
    "name": "tavily",
    "type": "tool_call",
}
tool.invoke(model_generated_tool_call)

ToolMessage(content=[{'url': 'https://www.radiotimes.com/tv/sport/football/euro-2024-location/', 'content': "Euro 2024 host cities. Germany have 10 host cities for Euro 2024, topped by the country's capital Berlin. Berlin. Cologne. Dortmund. Dusseldorf. Frankfurt. Gelsenkirchen. Hamburg."}, {'url': 'https://www.sportingnews.com/ca/soccer/news/list-euros-host-nations-uefa-european-championship-countries/85f8069d69c9f4ecd00c4900', 'content': 'The 2024 UEFA European Championship, more commonly known as Euro 2024, will mark the 17th edition of the tournament and is set to be hosted by Germany from June 14 to July 14.'}, {'url': 'https://footballgroundguide.com/euro-2024-host-nation', 'content': 'As the Euro 2024 host nation, Germany will play the opening match against Scotland on 14 June before clashing with Hungary and finally Switzerland in the group stages. It is not an easy group but Germany is the highest-ranked by FIFA in group A of the Euro 2024. Apart from European Championship acc

## Chaining

We can use our tool in a chain by first binding it to a [tool-calling model](/docs/how_to/tool_calling/) and then calling it:

```{=mdx}
import ChatModelTabs from "@theme/ChatModelTabs";

<ChatModelTabs customVarName="llm" />
```

In [16]:
# | output: false
# | echo: false

# !pip install -qU langchain langchain-openai
from langchain.chat_models import init_chat_model

llm = init_chat_model(model="gpt-4o", model_provider="openai")

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableConfig, chain

prompt = ChatPromptTemplate(
    [
        ("system", "You are a helpful assistant."),
        ("human", "{user_input}"),
        ("placeholder", "{messages}"),
    ]
)

# specifying tool_choice will force the model to call this tool.
llm_with_tools = llm.bind_tools([tool], tool_choice=tool.name)

llm_chain = prompt | llm_with_tools


@chain
def tool_chain(user_input: str, config: RunnableConfig):
    input_ = {"user_input": user_input}
    ai_msg = llm_chain.invoke(input_, config=config)
    tool_msgs = tool.batch(ai_msg.tool_calls, config=config)
    return llm_chain.invoke({**input_, "messages": [ai_msg, *tool_msgs]}, config=config)


tool_chain.invoke("who won the last womens singles wimbledon")

Here's the [LangSmith trace](...) for this run.

## API reference

For detailed documentation of all TavilySearchResults features and configurations head to the API reference: https://api.python.langchain.com/en/latest/tools/langchain_community.tools.tavily_search.tool.TavilySearchResults.html